In [3]:
# import các thư viện cần thiết
import gym_super_mario_bros as mario
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
import random as random
from matplotlib import pyplot as plt
import os
import copy
import time

In [4]:
# cài đặt các tham số của thuật toán di truyền, danh sách tham số đã được liệt kê trong báo cáo
ga_database = 'GA_database' 
init_size = 50
pop_size = 50
time_remainning = 400
mutate1_prob = 0.05
mutate2_prob = 0.02
mutate3_prob = 0.05
mutate4_prob = 0.05
crossover_prob = 0.1
stack_frame = 4
status = {"small" : 0, "tall" : 1, "fireball" : 2}

In [5]:
# Class các cá thể trong quần thể, evaluate() dùng để đánh giá cá thể theo mô tả trong báo cáo
class Individual:
    # hàm khơi tạo cá thể bằng cách khởi tạo ngẫu nhiên
    def __init__(self, actions=None, fitness =None, score = None, dis = None):
        self.fitness = 0
        self.score = 0
        self.dis = 0
        if actions == None:
            actions = []
            for i in range(init_size):
                action = random.randrange(0, len(SIMPLE_MOVEMENT))
                rep = random.randrange(1, 12)
                for j in range(rep):
                    actions.append(action)
            self.actions = actions
            self.evaluate()
        else:
            if fitness == None:
                self.actions = actions
                self.evaluate()
            else:
                self.actions = actions
                self.fitness = fitness
                self.score = score
                self.dis = dis
                
    def evaluate(self):
        fitness = 0
        tmp_actions = []
        cur_status = 0
        reward = 0
        done = 0 
        info = 0
        state = env.reset()
        for action in self.actions:
            for qq in range(stack_frame):
                state, reward, done, info = env.step([action])
                # nếu mario về đích sẽ cắt hành động thừa phía sau
                if info[0]['flag_get'] == True:
                    tmp_actions.append(action)
                    fitness += reward[0]
                    self.dis = info[0]['x_pos']
                    self.score = info[0]['score']
                    self.actions = tmp_actions
                    self.fitness = fitness + info[0]['score'] * 5
                    return 
                # nếu mario bị chết trong quá trình di chuyển thì sẽ xóa toàn bộ hành động phía sau hành động chết và xóa tiếp 24 hành động liền trước
                if  'terminal_observation' in info[0] or status[info[0]['status']] < cur_status:
                    for i in range(24):
                        if len(tmp_actions) > 1:
                            del tmp_actions[-1]
                    self.actions = tmp_actions
                    self.evaluate()
                    return
                cur_status = status[info[0]['status']]
                fitness += reward[0]
            tmp_actions.append(action)
        self.dis = info[0]['x_pos'] 
        self.score = info[0]['score']
        self.actions = tmp_actions
        # hàm fitness được cái đặt
        self.fitness = fitness + info[0]['score'] * 5

In [6]:
# class quần thể
class Population:
    def __init__(self, save_file = "", gen = 0):
        self.pop = []
        self.gen = 0
        if(save_file == ""):
            for i in range(0, pop_size):
                print(i)
                self.pop.append(Individual())
        else:
            self.loadFileToPop(save_file, gen)
    
    #một bước tiến hóa của quần thể
    def evolve(self):
        for ind in self.pop:
            print(ind.fitness, ind.score, ind.dis)
        print("--------------------------------------------")
        length = len(self.pop)
        
        # thực hiện chọn lọc
        
        for i in range(0, length - pop_size):
            del self.pop[-1]
            
        # chọn 2 cá thể ưu tiên cho đột biến
        
        a = random.randrange(0, int(pop_size / 10))
        while True:
            b = random.randrange(0, int(pop_size / 5))
            if b != a:
                break
        self.pop[a] = self.mutate2(self.pop[a])
        self.pop[a] = self.mutate3(self.pop[a])
        self.pop[b] = self.mutate2(self.pop[b])
        self.pop[b] = self.mutate3(self.pop[b])
        
        # đột biến quần thể
        
        for i, ind in enumerate(self.pop):
            alpha = random.random()
            if alpha <= mutate1_prob:
                self.pop[i] = self.mutate1(self.pop[i])
            elif alpha <= mutate1_prob + mutate2_prob:
                self.pop[i] = self.mutate2(self.pop[i])
            elif alpha <= mutate1_prob + mutate2_prob + mutate3_prob:
                self.pop[i] = self.mutate3(self.pop[i])
            elif alpha <= mutate1_prob + mutate2_prob + mutate3_prob + mutate4_prob:
                self.pop[i] = self.mutate4(self.pop[i])   
                
        # lai ghép các cá thể
        
        for i in range(int(crossover_prob * pop_size)):
            a = random.randrange(0, int(pop_size / 5))
            while True:
                b = random.randrange(0, pop_size)
                if b != a:
                    break
            ind1, ind2 = self.crossOver(copy.deepcopy(self.pop[a]), copy.deepcopy(self.pop[b]))
            self.pop.append(ind1)
            self.pop.append(ind2)
         
        # sắp xếp theo thứ tự fitness
            
        self.pop.sort(key=lambda x: x.fitness, reverse=True)
        self.gen += 1

    def crossOver(self, ind1, ind2):
        child1 = []
        child2 = []
        cut_ind = random.randrange(0, min(len(ind1.actions), len(ind2.actions)))
        for i in range(0, cut_ind):
            child1.append(ind1.actions[i])
            child2.append(ind2.actions[i])
        child1.append(int((ind1.actions[cut_ind] + ind2.actions[cut_ind]) / 2))
        child2.append(int((ind1.actions[cut_ind] + ind2.actions[cut_ind]) / 2))
        for i in range(cut_ind + 1, len(ind2.actions)):
            child1.append(ind2.actions[i])
        for i in range(cut_ind + 1, len(ind1.actions)):
            child2.append(ind1.actions[i])
        return Individual(child1), Individual(child2)

    # chen vao giua chuoi hanh dong
    def mutate1(self, ind):
        t = copy.deepcopy(ind)
        start = random.randrange(0, len(t.actions))
        action = random.randrange(0, len(SIMPLE_MOVEMENT))
        rep = random.randrange(1, 12)
        actions = t.actions[:start] + [action for i in range(rep)] + t.actions[start:]
        x = Individual(actions)
        del t
        if x.fitness > ind.fitness:
            return x
        else:
            return ind

    # chen vao cuoi chuoi hanh dong
    def mutate2(self, ind):
        t = copy.deepcopy(ind)
        res = []
        res2 = []
        rep = random.randrange(1, 12)
        for action in range(1, len(SIMPLE_MOVEMENT) - 1):
            if(action == 2):
                continue
            if(action == 4 and (ind.actions[-1] == 4 or ind.actions[-1] == 2)):
                actions = t.actions + [0] + [action for i in range(rep)]
            else:
                actions = t.actions + [action for i in range(rep)]
            x = Individual(actions)
            print(action, x.fitness)
            if x.fitness > t.fitness:
                res.append(x)
            if x.fitness == t.fitness:
                res2.append(x)
        if len(res) == 0:
            if(len(res2) == 0):
                for i in range(24):
                    if len(t.actions) > 1:
                        del t.actions[-1]
                t.evaluate()
                return t
            else:
                return res2[random.randrange(0, len(res2))]
        return res[random.randrange(0, len(res))]
            
    # chen nhay vao cuoi chuoi hanh dong
    def mutate3(self, ind):
        t = copy.deepcopy(ind)
        rep = random.randrange(4, 12)
        actions = t.actions + [0]
        actions = actions + [4 for i in range(rep)]
        x = Individual(actions)
        del t
        if x.fitness > ind.fitness:
            return x
        else:
            return ind   
            
    # chen chuoi vao giua
    def mutate4(self, ind):
        t = copy.deepcopy(ind)
        rep = random.randrange(1, 12)
        action = random.randrange(0, len(SIMPLE_MOVEMENT))
        start = random.randrange(0, len(t.actions))
        actions = t.actions
        for i in range(start, min(len(actions), start + rep)):
            actions[i] = action
        x = Individual(actions)
        if x.fitness > ind.fitness:
            return x
        else:
            return ind
    
    def savePopToFile(self, save_file):
        save = open(save_file, 'w')
        for ind in self.pop:
            for j in ind.actions:
                save.write(str(j) + ' ')
            save.write(str(ind.fitness) + ' ' + str(ind.score) + ' ' + str(ind.dis) + '\n')
        save.close()
    
    def loadFileToPop(self, save_file, gen):
        self.gen = gen
        self.pop = []
        save = open(save_file, 'r')
        lines = save.readlines()
        for line in lines:
            tmp = tuple(map(float, line.split(" ")[:]))
            actions = []
            for i in range(0, len(tmp) - 3):
                actions.append(int(tmp[i]))
            fitness = tmp[-3]
            score = tmp[-2]
            dis = tmp[-1]
            self.pop.append(Individual(actions, fitness, score, dis))
        save.close()


In [58]:
# cài đặt môi trường game và chọn màn chơi. Nếu muốn thay đổi màn chơi thì thay đổi ở đây (ví dụ 1-1, 1-2, 1-3, 1-4, 2-1, 2-2, 2-3, 2-4)
name_map = "1-1" 
env = mario.make('SuperMarioBros-' + name_map + '-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')
random.seed(0)

# tải lên quần thể tiến hóa nhất từ kho dữ liệu
listGaFile = os.listdir(ga_database + '/mario_ga' + name_map)
pops = []
for file in listGaFile:
    g = ""
    for ch in file:
        if ch != '.':
            g = g + ch
        else:
            break
    g = int(g)
    try:
        pops.append(Population(ga_database + '/mario_ga' + name_map + '/' + file, g))
    except:
        pass
pops.sort(key=lambda x: x.gen)
pop = pops[-1]
# quan sát cá thể tốt nhất hiện tại chơi game
x = copy.deepcopy(pop.pop[0])
env.reset()
for id in range(0, len(x.actions)):
    for qq in range(stack_frame):
        action = x.actions[id]
        state, reward, done, info = env.step([action])
        time.sleep(0.0001)
        env.render()

In [23]:
# tiến hóa tiếp, nếu không muốn train mà chỉ muốn xem các quần thể hiện tại thì không cần chạy
for i in range(0, 1):
    pop.evolve()
    state = env.reset()
    for id in range(0, len(pop.pop[0].actions)):
        for qq in range(stack_frame):
            action = pop.pop[0].actions[id]
            state, reward, done, info = env.step([action])
            env.render()
    pop.savePopToFile(ga_database + '/mario_ga' + name_map + '/' + str(pop.gen) + '.txt')

60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60234.0 11450.0 3161.0
60231.0 11450.0 3161.0
60223.0 11450.0 3161.0
60223.0 11450.0 3161.0
60223.0 11450.0 3161.0
60223.0 11450.0 3161.0
60223.0 11450.0 3161.0
60223.0 11450.0 3161.0
59993.0 11450.0 2921.0
59987.0 11450.0 2914.0
59973.0 11400.0 3161.0
59973.0 11400.0 3161.0
59737.0 11400.0 2914.0
59732.0 11350.0 3161.0
59731.0 11350.0 3161.0
59731.0 11350.0 3161.0
59728.0 113